Einführung in SciPy und SymPy
=============================

Peter Hrenka, TÜBIX 2015

Repository auf [github](https://github.com/hrenkap/scipy_workshop)

1. IPython Grundlagen
------------------------

[IPython](http://ipython.org/) ist eine interaktive Shell für Python. Startet man IPython mit

`> ipython notebook`

wird ein lokaler Webserver gestartet, so daß man mit
einem Browser neue Notebooks erstellen und editieren kann.

Es wird das asynchrone Python-Webserver [tornado](http://www.tornadoweb.org/en/stable/) verwendet, der eine Websocket-Verbindung mit dem Webbrowser herstellt. 

Für die Darstellung mathematischer Formeln wird [MathJax](http://www.mathjax.org/) verwendet, welches einen nützlichen Teil der $\LaTeX$-Syntax versteht.

Das Notebook wird über ipython zu einem `Kernel` verbunden, der in einem separaten laufenden Prozess läuft und die Python-Befehle ausführt. Es sind auch Kernel für [Julia](http://julialang.org/), [R](http://www.r-project.org/) und [Clang](https://github.com/minrk/clingkernel) und viele andere vorhanden.

Ein Notebook besteht aus mehreren "Zellen" (Cells) die Python-Code oder
[Markdown](https://en.wikipedia.org/wiki/Markdown)-formatierten Text enthalten können.


In [ ]:
import numpy as np
print(np.__version__)
np

Code-Zellen enthalten eine auch mehrzeilige Eingabe.

Ausgeführt wird der Code in der Zelle mit `CTRL + RETURN`.

Eine neue Zelle erhält man mit `ALT + RETURN`.

Eventuelle Ausgaben auf `sys.stdout` oder `sys.stderr` erscheinen direkt nach der Eingabebereich,
und der letzte Ausdruck wird im gesonderten Ausgabebereich angezeigt.

Wie `vi` unterscheidet `ipython` einen Kommando- und Edit-Modus:
  * Im Kommando-Modus kann man mit den Pfeiltasten die Zellen wechseln, und etwa mit `dd` die aktelle Zelle löschen
  * Mit `RETURN` oder Maus-Doppelklick kommt man in den Edit-Modus, den man mit `ESC` oder `CTRL + RETURN` verlassen kann

In [ ]:
# Interaktive Hilfe
np?

2. [numpy](http://www.numpy.org/)
---------------

`numpy` ist eine Python-Bibliothek die effiziente Arrays und darauf operierende Funktionen, die in C, C++ oder FORTRAN implementiert sind, in Python verfügbar macht. Sie steht unter der BSD-Lizenz. Mittlerweile ist sie (konzeptionell) ein Bestandteil von [SciPy](http://www.scipy.org/), kann aber meist separat installiert werden.

`numpy` ist recht umfangreich und leider nicht sehr gut modularisiert, so dass man sich beim ersten Start nicht über einige Gedenksekunden wundern sollte.

### Das Klasse `array`

Das zentrale Objekt in `numpy` ist das `array` (in der Dokumentation auch oft noch `ndarray` genannt).
Es ist ein mehrdimensionaler Container für Elemente des gleichen Datentyps.

In [ ]:
np.array?

In [ ]:
a = np.array([1,2,3,4], dtype=np.float32)
a

In [ ]:
a.shape, a.dtype

In [ ]:
a.flags

In [ ]:
b = a.reshape( (2,2) )
b

In [ ]:
b.flags

Man hieran erkennen, dass `a` eigene Daten besitzt (`OWNDATA: True`) und `b` nicht.
Das spart natürlich Speicher, kann aber auch leicht zu schwer lokalisierbaren Fehlern führen,
denn bei Änderungen an einem Objekt ändert sich auch das andere.


In [ ]:
c = np.array( np.reshape(a, (2,2)) )
c.flags.owndata

Lineare Bereiche kann man mit `linspace(start, stop, num)` erstellen.
Man beachte, dass der Parameter `num` einen etwas willkürlichen Standardwert von 50 hat,
und man daher besser immer die gewünsche Anzahl eingeben sollte.

In [ ]:
lin = np.linspace(1.0, 42.0, 42)
lin

Arrays können über eckige Klammern indiziert werden. Pro Dimension kann die Slice-Syntax wie bei Python-Listen verwendet werden, `a[start:stop]` oder `a[start:stop:step]`. Der Wert dieses Ausdrucks ist wieder ein Array der entsprechenden Dimension.
Solche Ausdrücke können auch das Ziel einer Zuweisung sein.

In [ ]:
rect = np.reshape(lin, (6,7))
rect

In [ ]:
# Alle Einträge von Zeile 1
rect[1, :]

In [ ]:
# jeder zweite Eintrag von Spalte 3
rect[::2, 2]

In [ ]:
cuboid = np.reshape(lin, (2,3,7))
cuboid

Zusätzlich zu konkreten Zahlen und Slices kann bei `arrays` auch die `Ellipsis` ... verwendet werden (die Offenbar nur für `numpy` in die Python-Grammatik eingebaut wurde). Diese entspricht der maximalen möglichen Anzahl an `:`-Slices und darf auch nur einmal vorkommen (analog zu `::` in IPv6).

In [ ]:
cuboid[1,...]

In [ ]:
cuboid[...,0]

### Arithmetik mit `arrays`

In [ ]:
lin + 1

Was ist da passiert? Die `1` wurde zu jedem Array-Element hinzuaddiert!

Das Verhalten nennt sich `broadcasting`: Numpy versucht, die Dimension(en) der Operanden aneinander anzugleichen.

In diesem Fall wird die `1` als array aufgefasst und die Dimension auf `42` erweitert.

In [ ]:
np.broadcast(lin, 1).shape

In [ ]:
lin*lin

Im Regelfall sind alle Operationen in `arrays` komponentenweise.

## Übung (Arrays)
    a) Fülle das 2d-array mit einem Schachbrettmuster, wobei Schwarz=0, Weiß=1
    b) Schreibe 2 auf die Positionen der Diagonale

In [ ]:
# Schachbrett
chess = np.zeros((10,10))
chess

## Beispiel: Die Mandelbrot-Menge

Betrachte die Folgen $z_{n+1} = z_n + c$ für alle $c\in \mathbb{C}$, $-2 < \textrm{Re } c < 1$, $-1 < \textrm{Im } c < 1$

Die 'Mandelbrot-Menge' ist die Menge aller $c$, für die $z_n$ beschränkt bleibt.

Eine Python-Funktion, welche die Folge berechnet, kann man so schreiben:

In [ ]:
def mandel_series(c, numIter):
    series = np.zeros(shape=(numIter,), dtype=np.complex128)
    z = 0+0j
    for i in range(numIter):
        z = z*z + c
        series[i] = z
    return series
mandel_series(0.5+0.4j, 10)

Das funktioniert, ist aber wenig anschaulich.

3. matplotlib
----------

Im `ipython`-Umfeld ist es sinnvoll, `matplotlib` über die zugehörige Direktive einzubinden
und einige Abkürzungssymbole zu definieren.

Matplotlib ist eine Python-Bibliothek, die unter einer BSD-artigen Lizenz steht und eine Reihe von Backends (wxPython, GTK+, Qt, …) unterstützt. Sie bietet eine ähnliche Funktionalität wie Gnuplot an, verwendet aber Python-Syntax und hat eine Reihe von Abhängigkeiten, zuvorderst `numpy`.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib as mp
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = 8,6 # Größe in Inches (!!!)

Die Funktion `plt.plot` generiert einen Plot, der direkt im Notebook angezeigt wird.

In [ ]:
s1 = mandel_series(0.2+0.5j, 100)
plt.plot(s1.real, s1.imag)
plt.show()

In [ ]:
plt.plot(np.abs(s1));

In [ ]:
s1 = mandel_series(0.2+0.5j, 100)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1) # ncols, nrows, plot_number
plt.title("Folge der $z_n$")
plt.plot(s1.real, s1.imag)
plt.xlabel("$\mathrm{Re} (z)$")
plt.ylabel("$\mathrm{Im} (z)$")

plt.subplot(1,2,2)
plt.title("Betrag von $z_n$")
plt.plot(np.abs(s1))
plt.xlabel("Iteration")
plt.ylabel("$\mathrm{abs} (z)$")

plt.show()

In [ ]:
s1 = mandel_series(0.30-0.3j, 100) # oder vielleicht 0.39-0.3j ?
plt.plot(s1.real, s1.imag)
plt.show()

Damit können wir individuell feststellen, welche $c$s zur Mandelbrot-Menge gehören.

Aber vielleicht erkennt man ja ein Muster, wenn man mehrere $c$ gleichzeitig berechnet…

In [ ]:
c = np.linspace(-2, 1, 800)+np.linspace(-1, 1, 600).reshape(600,1)*(0+1j)
c.shape, c[0,0], c[599,799]

In [ ]:
def mandel(c):
    z = np.zeros_like(c)
    for i in range(10):
        z = z*z + c
    return z
z = mandel(c)
z[0,0]

### Performancemessung in `ipython`

Mit der `%timeit` Direktive kann man in IPython einfach die Geschwindigkeit einer Funktion messen:

In [ ]:
%timeit mandel(c)

In [ ]:
(abs(z)<100.0)

In [ ]:
ax = plt.imshow(np.log(abs(z)), extent=(-2,1,-1,1));
plt.colorbar(ax)
plt.show()

In [ ]:
plt.imshow(np.angle(z), extent=(-2,1,-1,1));

In [ ]:
def mandel2(c, numIter=10, maxabs=5):
    z = c
    num = np.zeros_like(c, dtype=np.int16)
    for i in range(numIter):
        mask = (abs(z)<maxabs)
        z = mask*(z**2 + c)
        num += (abs(z)<maxabs)
    return z, num

In [ ]:
z, num = mandel2(c, 100, 5)
plt.figure(figsize=(12,9))
plt.imshow(num, extent=(-2,1,-1,1));

In [ ]:
plt.imshow(num, extent=(-2,1,-1,1));
plt.axhline(0.2)
plt.axvline(-0.75)
plt.show()

## Übung matplotlib
  * Suche eine interessante Stelle im Apfelmännchen und markiere diese im Plot
  * Suche neue Grenzen um diese Stelle herum, berechene neu und zeichne die Vergrößerung
  * Bestimme und markiere einige Punkte in der Vergrößerung und plotte die zugehörigen Reihen

4. [SymPy](http://www.sympy.org/en/index.html)
--------

SymPy ist ein Computer Algebra System das in reinem Python geschrieben ist.
  * BSD Lizensiert, als Bibliothek in (kommerzieller) Software nutzbar
  * Leichtgewichtig (im Gegenastz zu [Sage](http://www.sagemath.org/)) 

In [ ]:
import sympy as sp
sp.init_printing()
a, b, x, y = sp.symbols("a b x y")
a, b, x, y

Die Variablen $a$, $b$, $x$ und $y$ sind nun symbolische Werte, mit denen man in Python-Syntax Ausdrücke und Gleichungen eingeben kann:

In [ ]:
greekSyms = sp.symbols("alpha beta gamma delta lamda Delta Sigma")
greekSyms

In [ ]:
alpha, beta, gamma = greekSyms[:3]

In [ ]:
f, g, h = sp.symbols("f g h", cls=sp.Function)
type(f)

In [ ]:
a+b, sp.diff(f(alpha)), sp.sqrt(a**2+b/gamma), sp.Eq(x+y,gamma), sp.Sum(x**a, (a, 1, b)), sp.Integral((1/x)**2,(x,1,sp.oo))

Für Ausdrücke wie Integrale, die eventuell ausgewertet werden können, kann man mit der Methode `.doit()` diese Auswertung anstossen:

In [ ]:
intExpr = sp.Integral((1/x)**2,(x,1,sp.oo))
sp.Eq(intExpr, intExpr.doit())

In [ ]:
rootExpr = sp.sqrt(a**2+b/gamma)
sp.Eq(sp.Derivative(rootExpr, a), rootExpr.diff(a))

*Merke*: Großgeschriebene Bezeichner (`Sum`, `Derivative`, `Integral`) erhalten die symbolische Operation,
kleingeschriebene (`sum`, `diff`, `integrate`) Verben führen die Operation gleich durch.

In SymPy-Ausdrücken kann man mit `sp.subst` konkrete Werte einsetzen:

In [ ]:
valDict = dict(a=sp.pi, b=sp.E)
(a+b).subs(valDict)

Wer es noch konkreter mag, kann `evalf` verwenden:

In [ ]:
(a+b).subs(valDict).evalf(100)

In [ ]:
binom = (a+b)**5
binom.doit()

`doit()` führt nur Berechnungen aus (wie Summen, Ableitungen und Integrale), für Umformungen stehen unter anderem Folgende Funktionen zur Verfügung:

In [ ]:
polynom = sp.expand(binom)
polynom

In [ ]:
sp.factor(polynom)

In [ ]:
sp.tan(x).rewrite(sp.sin)

Gleichungen kann man mittels `solve` nach einer Variable auflösen:

In [ ]:
quad = sp.Eq(x**2+a*x+b, 0)
sp.solve(quad, x)

In [ ]:
sp.sin(x).series(x,n=10) #.removeO()

### Zusammenspiel `sympy` und `numpy`

Symbolische Berechnungen sind schön und gut, aber auch recht langsam.
die Funktion `lambdify` kann einen `sympy`-Ausdruck in eine `numpy`-Funktion umwandeln, die man dann wie gehabt plotten kann.

In [ ]:
def toLaTeX(symExpr):
    return "$"+sp.latex(symExpr)+"$"

domain = np.linspace(-4, 4, 1000)
origFunc = sp.sin(x)
numFunc = sp.lambdify(x, origFunc, "numpy")
plt.plot(domain, numFunc(domain), label=toLaTeX(origFunc))
plt.ylim([-1.0, 1.0])
plt.axvline(x=0, color='black')
plt.axhline(y=0, color='black')
plt.legend()
plt.show()

In [ ]:
domain = np.linspace(-4, 4, 1000)
origFunc = sp.sin(x)
numFunc = sp.lambdify(x, origFunc, "numpy")
plt.plot(domain, numFunc(domain), label=toLaTeX(origFunc))
for i in range(11)[3::2]:
    symFunc = origFunc.series(x, n=i).removeO()
    numFunc = sp.lambdify(x, symFunc, "numpy")
    plt.plot(domain, numFunc(domain), label=toLaTeX(symFunc))
plt.ylim([-2.0, 2.0])
plt.axvline(x=0, color='black')
plt.axhline(y=0, color='black')
plt.legend(loc=(0.5,0))
plt.show()

### Übung Ableitungen von $\sin(x)$

Berechne die Abeitungen von $\sin(x)$ für $x\in\{-3, -2, -1, 0, 1, 2, 3\}$ und zeichne die Tangenten.

## Dateiformat von Notebooks

Die Notebooks werden im im [JSON](https://en.wikipedia.org/wiki/JSON)-Format in Dateien mit der Endung `.ipynb` gespeichert.

Diese lassen sich per 

`> ipython nbconvert --to latex|pdf|html` Dateiname.ipynb

nach LaTeX, PDF, HTML und weitere Formate exportieren.

Allerdings wird dafür eine funktionierende LaTeX-Installation sowie [pandoc](http://pandoc.org/) vorausgesetzt.

Bilder und Formeln werden in einer String-Codierung gespeichert, wodurch Notebook-Files schnell anwachsen können. Um die Dateien klein zu halten und reproduzierbare Ergebnisse zu erhalten, kann man vor dem Speichern über `Cell -> All Output -> clear` die Ausgaben löschen und nach dem Laden `Cell -> Run All` ausführen.